In [2]:
import spacy
import pandas as pd

In [47]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Gaussian networks are uncapable of doing so.")

#### Stemming and Lemmatization

In [48]:
token_list = [[token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
               token.shape_, token.is_alpha, token.is_stop] for token in doc]

df = pd.DataFrame(columns=['text', 'lemma', 'pos_', 'tag_', 'dep_', 'shape_', 'is_alpha', 'is_stop'], 
                  data=token_list)

df

,text,lemma,pos_,tag_,dep_,shape_,is_alpha,is_stop
0,Gaussian,gaussian,ADJ,JJ,amod,Xxxxx,True,False
1,networks,network,NOUN,NNS,nsubj,xxxx,True,False
2,are,be,AUX,VBP,ROOT,xxx,True,True
3,uncapable,uncapable,ADJ,JJ,acomp,xxxx,True,False
4,of,of,ADP,IN,prep,xx,True,True
5,doing,do,VERB,VBG,pcomp,xxxx,True,True
6,so,so,ADV,RB,advmod,xx,True,True
7,.,.,PUNCT,.,punct,.,False,False


In [50]:
spacy.explain('prep')

'prepositional modifier'

In [16]:
print("Pipeline:", nlp.pipe_names)
doc = nlp("I was reading the paper.")
token = doc[0]  # 'I'
print(token.morph)
print(token.morph.get("PronType"))

Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Case=Nom|Number=Sing|Person=1|PronType=Prs
['Prs']


In [37]:
from spacy.lang.en import English

nlp = English()  # just the language with no pipeline
nlp.add_pipe("sentencizer")
doc = nlp("This is a sentence. This is another sentence.")
for sent in doc:
    print(sent.text)

This
is
a
sentence
.
This
is
another
sentence
.


In [4]:
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Hello World!")
displacy.serve(doc, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
